In [9]:
%load_ext dotenv
%dotenv
import jax.numpy as jnp
import jax
import math
import sklearn # Only for downloading MNIST Dataset and Accuracy Metrics
import sklearn.metrics
from keras.utils import to_categorical  # Only for categorical one hot encoding
import random
import matplotlib.pyplot as plt
import tensorflow as tf

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [2]:
xor_x_train = jnp.array([[0, 0], [0, 1], [1, 0], [1, 1], [1, 0], [0, 0], [1, 1]])
xor_y_train = jnp.array([[0], [1], [1], [0], [1], [0], [0]])
xor_x_train.shape, xor_y_train.shape

((7, 2), (7, 1))

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# (x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
cy_train = jnp.array(to_categorical(y_train))
cy_test = jnp.array(to_categorical(y_test))

cx_train, cx_test = jnp.array(x_train.reshape(-1, 784)/255.), jnp.array(x_test.reshape(-1, 784)/255.)
cx_train.shape

(60000, 784)

In [4]:
import NeuroLab as nl

# Neural Network with SGD

In [5]:
class SGD_Optimizer(nl.Optimizer):
    def __init__(self, model, learning_rate, loss, gamma=1, delta=2):
        super().__init__(model, loss)
        self.learning_rate = learning_rate
        self.gamma = gamma
        self.delta = delta

    def train_step(self, x_batch, y_batch):
        predictions = self.model.forward(x_batch)
        loss_value = self.loss.forward(predictions, y_batch)
        grad_loss = self.loss.backward(predictions, y_batch)

        for layer in reversed(self.model.get_layers()):
            # Back propogate the loss to the layer
            grad_loss, deltas = layer.backward(grad_loss)
            if deltas != None:
                # Simple Stochastic Gradient Decent
                delta_weights, delta_biases = deltas
                deltas = (delta_weights * self.learning_rate, delta_biases * self.learning_rate)
            # Update the weights
            layer.update_parameters(deltas)

        return loss_value

    def on_epoch_end(self, epoch):
        # Decay Weight
        self.learning_rate *= self.gamma
        self.delta *= self.gamma

In [6]:
nl.set_random_key(4)
# MNIST
layers = [
    nl.Dense(784, 128),
    nl.ReLU(),
    nl.Dense(128, 10),
    # Softmax(),
    nl.Sigmoid(),
]

net = nl.NeuralNet(layers)

optimizer = SGD_Optimizer(net, learning_rate = 0.01, loss=nl.MeanSquaredError(), delta=0.95, gamma=1)
optimizer.fit((cx_train, cy_train), (cx_test, cy_test), epochs=10, batch_size=100, verbose=True)
# Infer on cx_test, cy_test
predictions = net.predict(cx_test)
preds = jnp.array(nl.antiCategorical(predictions))
expected = jnp.array(nl.antiCategorical(cy_test))

print(f'Accuracy: {sklearn.metrics.accuracy_score(expected, preds)}')
print(f'Confusion Matrix: {sklearn.metrics.confusion_matrix(expected, preds)}')
print(f'Classification Report: {sklearn.metrics.classification_report(expected, preds)}')

Epoch 0, Loss: 0.006026385455412694, test_acc: 0.9328
Epoch 1, Loss: 0.003194459505843832, test_acc: 0.9488
Epoch 2, Loss: 0.0035828069852161786, test_acc: 0.9579
Epoch 3, Loss: 0.002534642267921961, test_acc: 0.9655
Epoch 4, Loss: 0.004401777960821992, test_acc: 0.9672
Epoch 5, Loss: 0.0027347030422612078, test_acc: 0.9694
Epoch 6, Loss: 0.0015456122289155658, test_acc: 0.9696
Epoch 7, Loss: 0.0023166172257838994, test_acc: 0.9719
Epoch 8, Loss: 0.00196298924717613, test_acc: 0.9726
Epoch 9, Loss: 0.0009780029497950715, test_acc: 0.9739
Accuracy: 0.9739
Confusion Matrix: [[ 972    0    1    1    0    2    0    2    2    0]
 [   0 1121    2    3    0    1    2    1    5    0]
 [   5    2  999    6    4    0    2   10    4    0]
 [   0    0    4  988    1    4    0    7    6    0]
 [   1    0    4    0  959    0    3    1    3   11]
 [   3    1    0    8    0  869    7    1    1    2]
 [   8    3    0    0    3    8  932    0    4    0]
 [   1    9   15    3    0    0    0  994    4    

# Bayesian Networks

In [ ]:
class BayesianDense(Layer):
    def __init__(self, n_input, n_output, prior_std=1.0, posterior_std=0.1):
        self.n_input = n_input
        self.n_output = n_output
        self.prior_std = prior_std
        self.posterior_std = posterior_std

        self.W_mu = jnp.random.normal(0, scale=(1/float(math.sqrt(n_input))), size=(n_input, n_output)).astype(jnp.float64)
        self.b_mu = jnp.zeros((1, n_output)).astype(jnp.float64)

        self.W_sigma = jnp.full((n_input, n_output), jnp.log(self.posterior_std)).astype(jnp.float64)
        self.b_sigma = jnp.full((1, n_output), jnp.log(self.posterior_std)).astype(jnp.float64)
        print("W_mu", jnp.mean(self.W_mu),
              "b_mu", jnp.mean(self.b_mu),
              "W_sigma", jnp.mean(self.W_sigma), self.W_sigma.shape,
              "b_sigma", jnp.mean(self.b_sigma), self.b_sigma.shape)

    def sample_weights(self):
        weights_std = jnp.exp(0.5 * self.W_sigma)
        self.W = self.W_mu + weights_std * jnp.random.normal(size=(self.n_input, self.n_output))
        biases_std = jnp.exp(0.5 * self.b_sigma)
        self.b = self.b_mu + biases_std * jnp.random.normal(size=(1, self.n_output))
        # print("w", jnp.mean(self.W), jnp.mean(weights_std), "b", jnp.mean(self.b), jnp.mean(biases_std))

    def forward(self, inputs):
        self.inputs = inputs
        self.sample_weights()
        self.output = jnp.dot(inputs, self.W) + self.b
        return self.output

    def backward(self, grad_output, learning_rate):
        grad_input = jnp.dot(grad_output, self.W.T)

        grad_W = jnp.dot(self.inputs.T, grad_output)
        grad_b = jnp.mean(grad_output, axis=0, keepdims=True)

        grad_W_mu = grad_W / grad_output.shape[0]
        grad_b_mu = grad_b / grad_output.shape[0]

        grad_W_sigma = ((grad_W ** 2) - 1) / (2 * grad_output.shape[0])
        grad_b_sigma = ((grad_b_mu ** 2) - 1) / (2 * grad_output.shape[0])

        self.W_mu -= learning_rate * grad_W_mu
        self.b_mu -= learning_rate * grad_b_mu

        self.W_sigma -= learning_rate * grad_W_sigma
        self.b_sigma -= learning_rate * grad_b_sigma

        return grad_input

In [ ]:
nl.set_random_key(4)
# MNIST
layers = [
    BayesianDense(784, 100),
    nl.LeakyReLU(),
    BayesianDense(100, 50),
    nl.LeakyReLU(),
    BayesianDense(50, 10),
    nl.Softmax(),
]

net = nl.NeuralNet(layers, learning_rate = 0.01, loss=nl.MeanSquaredError(), delta=0.95, gamma=1)

net.fit(cx_train, cy_train, epochs=10, batch_size=100, verbose=True)
# Infer on cx_test, cy_test
predictions = net.predict(cx_test, 20)
preds = jnp.array(antiCategorical(predictions)).get()
expected = jnp.array(antiCategorical(cy_test)).get()


print(f'Accuracy: {sklearn.metrics.accuracy_score(expected, preds)}')
print(f'Confusion Matrix: {sklearn.metrics.confusion_matrix(expected, preds)}')
print(f'Classification Report: {sklearn.metrics.classification_report(expected, preds)}')

W_mu -0.0001789835031751473 b_mu 0.0 W_sigma -2.3025850929940455 (784, 100) b_sigma -2.3025850929940455 (1, 100)
W_mu 0.0011180262676438772 b_mu 0.0 W_sigma -2.3025850929940455 (100, 50) b_sigma -2.3025850929940455 (1, 50)
W_mu 0.0023444765023452477 b_mu 0.0 W_sigma -2.302585092994045 (50, 10) b_sigma -2.3025850929940455 (1, 10)
Epoch 0, Batch 0, Loss: 0.07363589103305021
Epoch 1, Batch 0, Loss: 0.03985098104954909
Epoch 2, Batch 0, Loss: 0.02593534829055494
Epoch 3, Batch 0, Loss: 0.02348652026844813
Epoch 4, Batch 0, Loss: 0.019979116766026435
Epoch 5, Batch 0, Loss: 0.01774217933964233
Epoch 6, Batch 0, Loss: 0.017500500468994637
Epoch 7, Batch 0, Loss: 0.01771578663478224
Epoch 8, Batch 0, Loss: 0.01647555074216599
Epoch 9, Batch 0, Loss: 0.013828134710165842
Accuracy: 0.7548
Confusion Matrix: [[ 866    0   10    1    1   11    3    2   13   73]
 [   0 1078    2    4    0    0    4    0   44    3]
 [  25   15  810   20    9    5   20   13   76   39]
 [  19    3   36  587    0  197 

In [ ]:
predictions = net.predict(cx_test, 100)
preds = jnp.array(antiCategorical(predictions)).get()
expected = jnp.array(antiCategorical(cy_test)).get()

print(f'Accuracy: {sklearn.metrics.accuracy_score(expected, preds)}')
print(f'Confusion Matrix: {sklearn.metrics.confusion_matrix(expected, preds)}')
print(f'Classification Report: {sklearn.metrics.classification_report(expected, preds)}')

Accuracy: 0.6333
Confusion Matrix: [[747   0  19   1   0  16  84   1 111   1]
 [  0 982  22   7   0   0   3  27  94   0]
 [  4  54 706   5   6   8  41   1 207   0]
 [  9  37  40 685   2   9   8   2 217   1]
 [  4  17   6 122 518  34  17   4 243  17]
 [ 23  31  32  46   6 263  20   4 461   6]
 [  9   2  15   8   3   6 814   0 101   0]
 [  2   8  62  19   7  25   5 672 203  25]
 [  4  11  32   0   2   1  13   1 910   0]
 [  2   4  21 361   5   5   3 185 387  36]]
Classification Report:               precision    recall  f1-score   support

           0       0.93      0.76      0.84       980
           1       0.86      0.87      0.86      1135
           2       0.74      0.68      0.71      1032
           3       0.55      0.68      0.61      1010
           4       0.94      0.53      0.68       982
           5       0.72      0.29      0.42       892
           6       0.81      0.85      0.83       958
           7       0.75      0.65      0.70      1028
           8       0.31 

# Simulated Anealing

In [10]:

class SimulatedAnnealingOptimizer(nl.Optimizer):
    def __init__(self, model, learning_rate, loss):
        super().__init__(model, loss)
        self.learning_rate = learning_rate
        self.best_loss = float("inf")

    def forward_with_perturbations(self, x_train, stddev):
        perturbations = []
        inputs = x_train
        for layer in self.model.get_layers():
            if isinstance(layer, nl.SimulatedAnnealingLayer):
                perturbation = layer.perturb_parameters(stddev)
                perturbations.append(perturbation)
                inputs = layer.forward_with_perturbations(inputs, perturbation)
            else:
                inputs = layer.forward(inputs)
        return inputs, perturbations

    def update_parameters(self, perturbations):
      i = 0
      for layer in self.model.get_layers():
            if isinstance(layer, nl.SimulatedAnnealingLayer):
                layer.update_parameters(perturbations[i])
                i += 1

    def cooling_schedule(self, old_loss, new_loss, current_temp, current_step_size, cooling_rate, step_decay_rate=0.9999):
        deltaLoss = new_loss - old_loss
        deltaLossScale = deltaLoss / new_loss

        if abs(deltaLossScale) > min(1e-4, current_temp):
            scalefactor = min(1, current_temp/new_loss)
            elastic_rate = 1.0 - scalefactor
            elastic_rate = (elastic_rate + scalefactor) / 2
            rate = max(cooling_rate, elastic_rate)
            new_temp = current_temp * rate
            # new_temp = current_temp * cooling_rate
            new_step_size = current_step_size * step_decay_rate
        else:
            new_temp = current_temp / cooling_rate
            new_step_size = self.learning_rate
            self.learning_rate *= step_decay_rate

        return new_temp, new_step_size

    def train_step(self, x_batch, y_batch):
        for j in range(self.sample_per_batch):
            predictions, perturbations = self.forward_with_perturbations(x_batch, self.step_size)
            loss = self.loss.forward(predictions, y_batch)
            delta_E = (loss - self.best_loss)
            energy_cost = jnp.exp(-delta_E / self.current_temp)
            acceptance_rate = energy_cost if delta_E > 0 else  1.0
            if delta_E < 0 or jax.random.uniform(nl.get_random_key()) <= acceptance_rate:
                # Accept proposal
                self.update_parameters(perturbations)
                self.best_loss = loss
                # print(f'Accepted proposal due to temperature {loss} exp: {acceptance_rate}, deltaE {delta_E}')
        return self.best_loss

    def on_epoch_start(self, epoch):
        self.old_loss = self.best_loss

    def on_epoch_end(self, epoch):
        self.current_temp, self.step_size = self.cooling_schedule(self.old_loss, self.best_loss, self.current_temp, self.step_size, self.cooling_rate, 0.9999)

    def on_reporting(self, epoch, loss_value, acc):
        print(f'Epoch {epoch}, BestLoss: {loss_value}, Temperature {self.current_temp}, step_size {self.step_size}, test_acc: {acc}')

    def on_train_start(self, sample_per_batch=1, initial_temp=1.0, cooling_rate=0.99):
        self.current_temp = initial_temp
        # self.best_loss = float("inf")
        self.step_size = self.learning_rate
        self.sample_per_batch = sample_per_batch
        self.cooling_rate = cooling_rate

In [11]:
# XOR
nl.set_random_key(4)
layers = [
    nl.SADense(2, 3),
    nl.Sigmoid(),
    nl.SADense(3, 1),
    nl.Sigmoid(),
]

net = nl.NeuralNet(layers)
optimizer = SimulatedAnnealingOptimizer(net, learning_rate = 0.1, loss=nl.MeanSquaredError())

optimizer.fit((xor_x_train, xor_y_train), (xor_x_train, xor_y_train), epochs=1000, batch_size=60000, sample_per_batch=20, initial_temp=1.0, cooling_rate=0.95, verbose=True)
# Infer on cx_test, cy_test
preds = net(xor_x_train)
expected = xor_y_train

print("expected", expected)
print("preds", preds)

Epoch 0, BestLoss: 0.12472543282740714, Temperature 1.0, step_size 0.1, test_acc: 1.0
Epoch 1, BestLoss: 0.12504375122918168, Temperature 0.95, step_size 0.09999000000000001, test_acc: 1.0
Epoch 2, BestLoss: 0.15363896281138026, Temperature 0.9025, step_size 0.09998000100000001, test_acc: 1.0
Epoch 3, BestLoss: 0.13039663762975492, Temperature 0.8573749999999999, step_size 0.09997000299990001, test_acc: 1.0
Epoch 4, BestLoss: 0.1471675761057778, Temperature 0.8145062499999999, step_size 0.09996000599960002, test_acc: 1.0
Epoch 5, BestLoss: 0.2001160246950294, Temperature 0.7737809374999999, step_size 0.09995000999900006, test_acc: 1.0
Epoch 6, BestLoss: 0.19976442404111375, Temperature 0.7350918906249998, step_size 0.09994001499800016, test_acc: 1.0
Epoch 7, BestLoss: 0.20820310697963498, Temperature 0.6983372960937497, step_size 0.09993002099650036, test_acc: 1.0
Epoch 8, BestLoss: 0.20691650078368626, Temperature 0.6634204312890623, step_size 0.09992002799440071, test_acc: 1.0
Epoch 

In [12]:
# MNIST
nl.set_random_key(4)
layers = [
    nl.SADense(784, 128),
    nl.ReLU(),
    nl.SADense(128, 10),
    nl.Softmax(),
]

net = nl.NeuralNet(layers)
optimizer = SimulatedAnnealingOptimizer(net, learning_rate = 1, loss=nl.MeanSquaredError())

optimizer.fit((cx_train, cy_train), (cx_test, cy_test), epochs=10000, batch_size=60000, sample_per_batch=1, initial_temp=1.0, cooling_rate=0.95, verbose=True)
# Infer on cx_test, cy_test
predictions = net.forward(cx_test)
preds = jnp.array(nl.antiCategorical(predictions))
expected = jnp.array(nl.antiCategorical(cy_test))

print(f'Accuracy: {sklearn.metrics.accuracy_score(expected.get(), preds.get())}')
print(f'Confusion Matrix: {sklearn.metrics.confusion_matrix(expected.get(), preds.get())}')
print(f'Classification Report: {sklearn.metrics.classification_report(expected.get(), preds.get())}')

Epoch 0, BestLoss: 0.08846413571159746, Temperature 1.0, step_size 1, test_acc: 0.1062
Epoch 1, BestLoss: 0.08872017271870125, Temperature 0.95, step_size 0.9999, test_acc: 0.1098
Epoch 2, BestLoss: 0.08393766354802341, Temperature 0.9025, step_size 0.9998000100000001, test_acc: 0.1572
Epoch 3, BestLoss: 0.08797428968054603, Temperature 0.8573749999999999, step_size 0.9997000299990001, test_acc: 0.1163
Epoch 4, BestLoss: 0.08848969932164755, Temperature 0.8145062499999999, step_size 0.9996000599960002, test_acc: 0.1067
Epoch 5, BestLoss: 0.08799215453884722, Temperature 0.7737809374999999, step_size 0.9995000999900007, test_acc: 0.1199
Epoch 6, BestLoss: 0.08608629351141167, Temperature 0.7350918906249998, step_size 0.9994001499800017, test_acc: 0.1372
Epoch 7, BestLoss: 0.08783979435639122, Temperature 0.6983372960937497, step_size 0.9993002099650037, test_acc: 0.1208
Epoch 8, BestLoss: 0.09009245373456257, Temperature 0.6634204312890623, step_size 0.9992002799440072, test_acc: 0.096


# Extreme Learning Machines

In [14]:
# Extreme Learning Machines
class ELM_Optimizer(nl.Optimizer):
    def __init__(self, model, learning_rate, loss):
        super().__init__(model, loss)
        self.learning_rate = learning_rate

    def train_step(self, x_batch, y_batch, alpha=0):
        predictions = self.model.forward(x_batch)
        loss_value = self.loss.forward(predictions, y_batch)

        # Smooth out y_batch
        y_batch = jnp.where(y_batch > 0.5, 0.9, 0.1)
        expected = y_batch
        for layer in reversed(self.model.get_layers()):
            if isinstance(layer, nl.Dense):
                x_inv = jnp.linalg.pinv(layer.last_inputs)
                weight_approx = jnp.dot(x_inv, expected)
                layer.weights = layer.weights * alpha + weight_approx * (1 - alpha)
                # layer.weights = weight_approx
                # expected = layer.last_inputs
                expected = layer.inverse(expected)
            else:
                expected = layer.inverse(expected)
        return loss_value



In [86]:
# XOR
nl.set_random_key(4)
layers = [
    nl.Dense(2, 3, False),
    nl.LeakyReLU(),
    nl.Dense(3, 1, False),
    nl.LeakyReLU(),
]

net = nl.NeuralNet(layers)
optimizer = ELM_Optimizer(net, learning_rate = 0.1, loss=nl.MeanSquaredError())

optimizer.fit((xor_x_train, xor_y_train), (xor_x_train, xor_y_train), epochs=10, batch_size=60000, verbose=True)
# Infer on cx_test, cy_test
preds = net(xor_x_train)
expected = xor_y_train

print("expected", expected)
print("preds", preds)

Epoch 0, Loss: 0.16489561130438207, test_acc: 1.0
Epoch 1, Loss: 0.14321428571428574, test_acc: 1.0
Epoch 2, Loss: 0.1432142857142857, test_acc: 1.0
Epoch 3, Loss: 0.1432142857142857, test_acc: 1.0
Epoch 4, Loss: 0.1432142857142857, test_acc: 1.0
Epoch 5, Loss: 0.14321428571428574, test_acc: 1.0
Epoch 6, Loss: 0.1432142857142857, test_acc: 1.0
Epoch 7, Loss: 0.1432142857142857, test_acc: 1.0
Epoch 8, Loss: 0.1432142857142857, test_acc: 1.0
Epoch 9, Loss: 0.1432142857142857, test_acc: 1.0
expected [[0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]]
preds [[0.   ]
 [0.05 ]
 [0.475]
 [0.525]
 [0.475]
 [0.   ]
 [0.525]]


In [114]:
# MNIST
nl.set_random_key(4)
layers = [
    nl.Dense(784, 128, False),
    # LeakyReLU(),
    # Linear(),
    # Dense(128, 128, False),
    # LeakyReLU(),
    # Dense(128, 10, False),
    # Sigmoid(),
]

net = nl.NeuralNet(layers)
optimizer = ELM_Optimizer(net, learning_rate = 0.01, loss=nl.MeanSquaredError())

optimizer.fit((cx_train, cy_train), (cx_test, cy_test), epochs=1, batch_size=60000, verbose=True)
# Infer on cx_test, cy_test
predictions = net.forward(cx_test)
preds = jnp.array(nl.antiCategorical(predictions))
expected = jnp.array(nl.antiCategorical(cy_test))

print(f'Accuracy: {sklearn.metrics.accuracy_score(expected.get(), preds.get())}')
print(f'Confusion Matrix: {sklearn.metrics.confusion_matrix(expected.get(), preds.get())}')
print(f'Classification Report: {sklearn.metrics.classification_report(expected.get(), preds.get())}')

Epoch 0, Loss: 0.10928068989420225, test_acc: 0.8509
Accuracy: 0.8509
Confusion Matrix: [[ 936    0    3    3    2    9   17    1    7    2]
 [   0 1108    2    2    1    1    5    1   14    1]
 [  18   59  805   29   15    0   38   24   39    5]
 [   5   19   24  875    1   19    9   20   24   14]
 [   0   24    8    4  866    5    9    2   14   50]
 [  20   15    6   77   16  632   22   14   66   24]
 [  19   10   10    0   19   21  870    0    9    0]
 [   5   40   18    6   22    1    1  868    5   62]
 [  16   56    9   32   26   47   15   11  736   26]
 [  19   11    3   15   69    0    1   67   11  813]]
Classification Report:               precision    recall  f1-score   support

           0       0.90      0.96      0.93       980
           1       0.83      0.98      0.89      1135
           2       0.91      0.78      0.84      1032
           3       0.84      0.87      0.85      1010
           4       0.84      0.88      0.86       982
           5       0.86      0.71

In [112]:
# Given matrices
a = cy_train[:30000]
b = cx_train[:30000]
c = cy_train[30000:]
d = cx_train[30000:]

# Randomly initialize x and y
x = jnp.random.rand(b.shape[1], c.shape[1])
y = jnp.random.rand(a.shape[0], c.shape[0])


In [115]:

# Calculate intermediate variables
# z1 = jnp.dot(b, x)
# z2 = jnp.dot(d, x)
z1 = b
z2 = d

# Form stacked matrices
A = jnp.vstack((a, c))
Z = jnp.vstack((z1, z2))

# Compute pseudoinverse and solve for y
Z_pseudo_inv = jnp.linalg.pinv(Z)
y = jnp.dot(Z_pseudo_inv, A)

# Substitute back to solve for x
# B = jnp.vstack((b, d))
# Z_prime = jnp.vstack((jnp.dot(b, x), jnp.dot(d, x)))
# B_pseudo_inv = jnp.linalg.pinv(B)
# x = jnp.dot(B_pseudo_inv, Z_prime)

# output = jnp.dot(cx_test, x)
output = jnp.dot(cx_test, y)

preds = jnp.array(nl.antiCategorical(output))
expected = jnp.array(nl.antiCategorical(cy_test))

print(f'Accuracy: {sklearn.metrics.accuracy_score(expected.get(), preds.get())}')
print(f'Confusion Matrix: {sklearn.metrics.confusion_matrix(expected.get(), preds.get())}')
print(f'Classification Report: {sklearn.metrics.classification_report(expected.get(), preds.get())}')

Accuracy: 0.8534
Confusion Matrix: [[ 942    0    2    2    1    7   15    2    7    2]
 [   0 1107    2    2    1    1    5    2   15    0]
 [  17   56  809   28   16    0   42   21   39    4]
 [   4   15   26  887    2   14    9   21   21   11]
 [   0   23    6    3  872    5   10    2   13   48]
 [  20   17    2   84   19  624   22   13   69   22]
 [  17    9   10    0   21   20  872    0    9    0]
 [   5   38   18    8   20    0    1  877    3   58]
 [  17   54    9   32   27   42   15   12  743   23]
 [  18   10    2   15   72    1    1   76   13  801]]
Classification Report:               precision    recall  f1-score   support

           0       0.91      0.96      0.93       980
           1       0.83      0.98      0.90      1135
           2       0.91      0.78      0.84      1032
           3       0.84      0.88      0.86      1010
           4       0.83      0.89      0.86       982
           5       0.87      0.70      0.78       892
           6       0.88      0.9

# ELM + Simulated Annealing Hybrid

In [82]:
# Extreme Learning Machines Hybrid
class ELM_SA_Optimizer(SimulatedAnnealingOptimizer):
    def __init__(self, model, learning_rate, loss):
        super().__init__(model, learning_rate, loss)

    def train_step(self, x_batch, y_batch, alpha=0.8):
        predictions = self.model.forward(x_batch)
        # loss_value = self.loss.forward(predictions, y_batch)

        # Smooth out y_batch
        y_batch = jnp.where(y_batch > 0.5, 0.9, 0.1)
        expected = y_batch
        for layer in reversed(self.model.get_layers()):
            if isinstance(layer, Dense):
                x_inv = jnp.linalg.pinv(layer.last_inputs)
                weight_approx = jnp.dot(x_inv, expected)
                layer.weights = layer.weights * alpha + weight_approx * (1 - alpha)
                # layer.weights = weight_approx
                # expected = layer.last_inputs
                expected = layer.inverse(expected)
            else:
                expected = layer.inverse(expected)

        loss_value = super().train_step(x_batch, y_batch)
        return loss_value



# K-BFGS Algorithm

In [ ]:
class Layer:
    def forward(self, inputs):
        raise NotImplementedError

    def backward(self, grad_output):
        raise NotImplementedError

class Dense(Layer):
    def __init__(self, n_input, n_output):
        self.weights = jnp.random.randn(n_input, n_output) * jnp.sqrt(2.0 / n_input)
        self.biases = jnp.zeros((1, n_output))

    def forward(self, inputs):
        self.inputs = inputs
        self.output = jnp.dot(inputs, self.weights) + self.biases
        return self.output

    def set_weights_biases(self, weights, biases):
        self.weights = weights
        self.biases = biases

    def get_params(self):
        return jnp.concatenate([self.weights.flatten(), self.biases.flatten()])

    def set_params(self, params):
        weight_size = self.weights.size
        self.weights = params[:weight_size].reshape(self.weights.shape)
        self.biases = params[weight_size:].reshape(self.biases.shape)

class Sigmoid:
    @staticmethod
    def forward(x):
        return 1 / (1 + jnp.exp(-x))

    @staticmethod
    def backward(x):
        sigmoid = Sigmoid.forward(x)
        return sigmoid * (1 - sigmoid)

class ReLU:
    @staticmethod
    def forward(x):
        return jnp.maximum(0, x)

    @staticmethod
    def backward(x):
        return jnp.where(x > 0, 1, 0)

class Softmax:
    @staticmethod
    def forward(x):
        exp_x = jnp.exp(x - jnp.max(x, axis=1, keepdims=True))
        return exp_x / jnp.sum(exp_x, axis=1, keepdims=True)

    @staticmethod
    def backward(x):
        pass

class MeanSquaredError:
    @staticmethod
    def forward(predictions, targets):
        return jnp.mean((predictions - targets)**2)

    @staticmethod
    def backward(predictions, targets):
        return 2 * (predictions - targets) / predictions.size

class NeuralNet:
    def __init__(self, layers, loss):
        self.layers = layers
        self.loss = loss

    def forward(self, inputs):
        for layer in self.layers:
            inputs = layer.forward(inputs)
        return inputs

    def get_params(self):
        """Returns all layer parameters as a single flattened array."""
        params = jnp.concatenate([layer.get_params() for layer in self.layers if isinstance(layer, Dense)])
        return params

    def set_params(self, params):
        """Sets all layer parameters from a single flattened array."""
        start_idx = 0
        for layer in self.layers:
            if isinstance(layer, Dense):
                layer_params_size = layer.get_params().size
                layer.set_params(params[start_idx:start_idx + layer_params_size])
                start_idx += layer_params_size

    def loss_and_grad(self, params, x_train, y_train):
        """Calculates loss and gradient based on provided parameters."""
        self.set_params(params)
        predictions = self.forward(x_train)

        loss_value = self.loss.forward(predictions, y_train)
        grad_loss = self.loss.backward(predictions, y_train)

        grad_params = []
        for layer in self.layers:
            if isinstance(layer, Dense):
                grad_weights = jnp.dot(layer.inputs.T, grad_loss)
                grad_biases = jnp.sum(grad_loss, axis=0, keepdims=True)

                grad_params.append(grad_weights.flatten())
                grad_params.append(grad_biases.flatten())

                grad_loss = jnp.dot(grad_loss, layer.weights.T)

        grad_params = jnp.concatenate(grad_params)
        return loss_value, grad_params

    def k_bfgs(self, x_train, y_train, epochs=1, verbose=True):
        """Trains the model using the K-BFGS algorithm."""
        params = self.get_params()

        # Initialize memory for Hessian approximation
        s_list, y_list = [], []

        for epoch in range(epochs):
            loss_value, grad_params = self.loss_and_grad(params, x_train, y_train)

            # Form Hessian approximation from s and y lists
            H = jnp.eye(len(params))

            for s, y in zip(s_list, y_list):
                rho = 1 / jnp.dot(y, s)
                V = H - rho * jnp.outer(jnp.dot(H, s), y)
                H = V + rho * jnp.outer(s, s)

            # Compute search direction
            direction = -jnp.dot(H, grad_params)

            # Line search to find suitable step size
            step_size = 1.0

            def line_search_loss(step):
                new_params = params + step * direction
                new_loss, _ = self.loss_and_grad(new_params, x_train, y_train)
                return new_loss

            # Try step size reductions until loss decreases
            while line_search_loss(step_size) >= loss_value:
                step_size *= 0.5
                if step_size < 1e-10:
                    break

            new_params = params + step_size * direction

            # Update s and y lists
            s = new_params - params
            new_loss, new_grad_params = self.loss_and_grad(new_params, x_train, y_train)
            y = new_grad_params - grad_params

            if len(s_list) >= 10:  # Limit to 10 recent updates
                s_list.pop(0)
                y_list.pop(0)

            s_list.append(s)
            y_list.append(y)

            params = new_params

            if verbose:
                print(f'Epoch {epoch}, Loss: {loss_value}')

        self.set_params(params)

def antiCategorical(arr):
    return jnp.argmax(arr, axis=1)

def main():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

    y_train_c = jnp.array(to_categorical(y_train))
    y_test_c = jnp.array(to_categorical(y_test))

    x_train = jnp.array(x_train.reshape(-1, 784) / 255.)
    x_test = jnp.array(x_test.reshape(-1, 784) / 255.)

    layers = [
        Dense(784, 128),
        ReLU(),
        Dense(128, 10),
        Softmax(),
    ]

    nn = NeuralNet(layers, loss=MeanSquaredError())

    nn.k_bfgs(x_train, y_train_c, epochs=10, verbose=True)

    predictions = nn.forward(x_test)

    preds = antiCategorical(predictions)
    expected = antiCategorical(y_test_c)

    print(f'Accuracy: {sklearn.metrics.accuracy_score(expected, preds)}')
    print(f'Confusion Matrix: {sklearn.metrics.confusion_matrix(expected, preds)}')
    print(f'Classification Report: {sklearn.metrics.classification_report(expected, preds)}')

main()


In [ ]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

# Data Preprocessing
def preprocess_data():

    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

    y_train_c = jnp.array(to_categorical(y_train))
    y_test_c = jnp.array(to_categorical(y_test))

    x_train = jnp.array(x_train.reshape(-1, 784) / 255.)
    x_test = jnp.array(x_test.reshape(-1, 784) / 255.)

    # Split into training and testing sets
    return x_train, x_test, y_train_c, y_test_c

# Utility functions
def relu(x):
    return jnp.maximum(0, x)

def relu_derivative(x):
    return jnp.where(x > 0, 1, 0)

# Mesh Neural Network Class
class MeshNeuralNetwork:
    def __init__(self, num_neurons, input_size, output_size, num_passes=3, learning_rate=0.001):
        self.num_neurons = num_neurons
        self.input_size = input_size
        self.output_size = output_size
        self.num_passes = num_passes
        self.learning_rate = learning_rate

        # Initialize neurons
        self.neurons = [None] * num_neurons
        self.connections = jnp.random.choice(
            jnp.arange(num_neurons),
            size=(num_neurons, num_neurons),
            replace=True
        )

        # Initialize weights and biases randomly
        self.weights = jnp.random.randn(num_neurons, num_neurons)
        self.biases = jnp.random.randn(num_neurons)

        # Select random input/output neurons
        self.input_neurons = jnp.random.choice(jnp.arange(num_neurons), size=input_size, replace=False)
        self.output_neurons = jnp.random.choice(jnp.arange(num_neurons), size=output_size, replace=False)

    def forward_pass(self, X):
        # Initialize neuron activations with input data
        self.neurons[self.input_neurons] = X

        # Propagate for multiple passes
        for _ in range(self.num_passes):
            for i in range(self.num_neurons):
                # Update each neuron activation based on connections
                inputs = self.weights[i, self.connections[i]]
                activation_input = jnp.dot(inputs, self.neurons[self.connections[i]]) + self.biases[i]
                self.neurons[i] = relu(activation_input)

        # Output activations
        return self.neurons[self.output_neurons]

    def backward_pass(self, X, y_true, y_pred):
        # Loss gradient (Cross-Entropy Loss derivative)
        loss_grad = y_pred - y_true

        # Backpropagate through output neurons first
        for i in self.output_neurons:
            # Compute gradients w.r.t weights and biases
            neuron_output = self.neurons[i]
            relu_grad = relu_derivative(neuron_output)
            grad_w = jnp.outer(loss_grad[i], self.neurons[self.connections[i]]) * relu_grad
            grad_b = loss_grad[i] * relu_grad

            # Update weights and biases
            self.weights[i, self.connections[i]] -= self.learning_rate * grad_w
            self.biases[i] -= self.learning_rate * grad_b

        # Backpropagate further into the network
        for i in reversed(range(self.num_neurons)):
            if i in self.output_neurons:
                continue

            neuron_output = self.neurons[i]
            relu_grad = relu_derivative(neuron_output)
            grad_w = jnp.outer(loss_grad[i], self.neurons[self.connections[i]]) * relu_grad
            grad_b = loss_grad[i] * relu_grad

            self.weights[i, self.connections[i]] -= self.learning_rate * grad_w
            self.biases[i] -= self.learning_rate * grad_b

    def fit(self, X_train, y_train, epochs):
        for epoch in range(epochs):
            for X_batch, y_batch in zip(X_train, y_train):
                y_pred = self.forward_pass(X_batch)
                self.backward_pass(X_batch, y_batch, y_pred)

            # Output training metrics
            train_loss, train_accuracy = self.evaluate(X_train, y_train)
            print(f"Epoch {epoch+1}/{epochs} - Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}")

    def evaluate(self, X, y):
        correct_predictions = 0
        total_loss = 0

        for X_sample, y_true in zip(X, y):
            y_pred = self.forward_pass(X_sample)

            # Compute loss
            loss = jnp.mean(-y_true * jnp.log(y_pred) - (1 - y_true) * jnp.log(1 - y_pred))
            total_loss += loss

            # Compute accuracy
            correct_predictions += jnp.argmax(y_pred) == jnp.argmax(y_true)

        return total_loss / len(X), correct_predictions / len(X)

# Main Training Loop
def main():
    X_train, X_test, y_train, y_test = preprocess_data()

    # Convert to arrays for easier manipulation
    X_train, X_test = jnp.array(X_train), jnp.array(X_test)
    y_train, y_test = jnp.array(y_train), jnp.array(y_test)

    # Initialize network with a suitable number of neurons, input size, and output size
    input_size, output_size = X_train.shape[1], y_train.shape[1]
    num_neurons = 2000  # Example number
    network = MeshNeuralNetwork(num_neurons=num_neurons, input_size=input_size, output_size=output_size, num_passes=3)

    # Train the network
    network.fit(X_train, y_train, epochs=10)

    # Evaluate on the test set
    test_loss, test_accuracy = network.evaluate(X_test, y_test)
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

if __name__ == "__main__":
    main()


TypeError: only integer scalar arrays can be converted to a scalar index